In [2]:
!pip install yt
!pip install f90nml

In [ ]:
import yt
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

In [6]:
from math import pi, sqrt
mp = 1.66e-24 #g/cm^3
kB = 1.3806488e-16 #erg K^-1
gamma = 5/3
miu0=4.0*pi

# Choose your mode number
n = 3

He_abundance = 0.1
unit_numberdensity = 1
unit_temperature = 1
unit_length = 1

unit_density=(1.0+4.0*He_abundance)*mp*unit_numberdensity
unit_pressure=(2.0+3.0*He_abundance)*unit_numberdensity*kB*unit_temperature
unit_velocity=sqrt(unit_pressure/unit_density)
unit_magneticfield=sqrt(miu0*unit_pressure)
unit_time=unit_length/unit_velocity

print("UNIT Time ", unit_time)

#choose the correct path here
ds = yt.load("/content/drive/MyDrive/B=0.009/16_modes_B_0.0090000.dat")

ds.fields

ad = ds.all_data()
density = ad["gas","density"]
MF_1 = ad["gas","magnetic_1"]
thermal_pressure = ad["gas","thermal_pressure"]
Vx = ad["gas","velocity_x"]
Vy = ad["gas","velocity_y"]
x = ad["amrvac","y"]

me = 0.5*((ad["b1"].to_ndarray())**2 + (ad["b2"].to_ndarray())**2)
ke= 0.5 * ((ad["m1"].to_ndarray()**2 + ad["m2"].to_ndarray()**2)/ad["rho"].to_ndarray())
inte = ad["e"].to_ndarray() - me - ke 
pressure = inte * (gamma-1)
vel_x = ad["m1"].to_ndarray()/ad["rho"].to_ndarray() 
vel_y = ad["m2"].to_ndarray()/ad["rho"].to_ndarray() 
rho = ad["rho"].to_ndarray()
rho1 = rho[-1] 
rho2 = rho[0] 
print("Thermal pressure ", pressure)
print("Magnetic pressure ", me)
print("M1 ", ad["b1"].to_ndarray())
print("M2 ", ad["b2"].to_ndarray())
beta_plasma = pressure/me
print("BETA PLASMA ", beta_plasma[0])
deltaV = 2*vel_x[0]
print("Delta V nondim  ", deltaV)
print("Delta V with units  ", deltaV*unit_velocity)

domain_x = ds.domain_right_edge[0] - ds.domain_left_edge[0]
print("NONDIM domain_x ", domain_x)

k = float(2*np.pi*n/domain_x )
Growth_rate_squared = (k**2)*(rho1/(rho1+rho2)) *( ( rho2/(rho1+rho2) )*(deltaV**2)- 4/(gamma*beta_plasma[0]*rho1)* pressure[0]*gamma )
print("sigma**2 ", Growth_rate_squared)
print("NONDIM growth rate ", (Growth_rate_squared)**0.5)

#https://arxiv.org/pdf/1907.12507.pdf
#cgs units     
import numpy as np
domain_x = max(x)
print("DOMAIN X ", domain_x)



B = float(MF_1[0])# G
Pressure_magnetic = float((B**2)/(8*np.pi) )
k = float(2*np.pi*n/domain_x )


rho_1 = float(density[-1]) # g/cm^3
print("rho1 with units ", rho_1)
rho_2 =float( density[0]) # g/cm^3
print("rho2 with units ", rho_2)
Pressure =  float(thermal_pressure[0]) # g/(cm*s**2)
Delta_V = 2*float(Vx[0] )# cm/s #Vy probably
print("Delta V ", Delta_V)
Gamma = 5/3

Plasma_beta = Pressure/Pressure_magnetic
print("Plasma beta ", Plasma_beta)

V1 = Vx[-1]
V2 = Vx[0]

# NOTE: SECOND TERM ON THE RIGHT HAND SIDE SHOULD BE DELETED WHEN ANALYSING THE CASE WITHOUT A MAGNETIC FIELD
Growth_rate_squared = (k**2)*(rho_1/(rho_1+rho_2)) *( ( rho_2/(rho_1+rho_2) )*(Delta_V**2)- 4/(Gamma*Plasma_beta*rho_1)* Pressure*Gamma  )
print("sigma**2 with units ", Growth_rate_squared)
Growth_rate = (Growth_rate_squared)**0.5

print(Growth_rate)
print("NONDIM growth rate ", Growth_rate*unit_time)
print(abs(Growth_rate))